In [20]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import json
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
# Load your CSV file
df = pd.read_csv("books_cleaned.csv")
df['description'] = df['description'].fillna('')
df['isbn13'] = df['isbn13'].astype(str)
df_clean = df[df['description'].str.len() > 0].copy()

# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"Loaded {len(df_clean)} books")


Loaded 5197 books


In [22]:
descriptions = df_clean['description'].tolist()
print("Generating embeddings...")
embeddings = model.encode(descriptions, show_progress_bar=True)

# Normalize embeddings for cosine similarity
embeddings_normalized = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

print(f"Embeddings shape: {embeddings.shape}")


Generating embeddings...


Batches: 100%|██████████| 163/163 [01:36<00:00,  1.69it/s]

Embeddings shape: (5197, 384)


In [23]:
# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product for cosine similarity
index.add(embeddings_normalized.astype('float32'))

print(f"FAISS index created with {index.ntotal} vectors")

FAISS index created with 5197 vectors


In [24]:
# Store metadata alongside FAISS index
metadata = []
for _, row in df_clean.iterrows():
    meta = {
        'isbn13': str(row['isbn13']),
        'title': str(row.get('title', '')),
        'authors': str(row.get('authors', '')),
        'categories': str(row.get('categories', '')),
        'description': str(row.get('description', '')),
        'published_year': str(row.get('published_year', '')),
        'average_rating': str(row.get('average_rating', '')),
        'num_pages': str(row.get('num_pages', '')),
        'ratings_count': str(row.get('ratings_count', ''))
    }
    metadata.append(meta)

print(f"Metadata prepared for {len(metadata)} books")

Metadata prepared for 5197 books


In [25]:
class BookSearchFAISS:
    def __init__(self, index, metadata, model):
        self.index = index
        self.metadata = metadata
        self.model = model
    
    def similarity_search(self, query: str, k: int = 10):
        # Encode query
        query_embedding = self.model.encode([query])
        query_normalized = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
        
        # Search
        scores, indices = self.index.search(query_normalized.astype('float32'), k)
        
        # Format results
        docs = []
        for i, idx in enumerate(indices[0]):
            doc = self.metadata[idx].copy()
            doc['similarity_score'] = float(scores[0][i])
            docs.append(doc)
        
        return docs
    
    def print_results(self, docs, max_desc_len=150):
        for i, doc in enumerate(docs, 1):
            print(f"{i}. 📚 {doc['title']}")
            print(f"   👤 Authors: {doc['authors']}")
            print(f"   🏷️  Category: {doc['categories']}")
            print(f"   ⭐ Rating: {doc['average_rating']} | 📄 Pages: {doc['num_pages']}")
            
            desc = doc['description']
            if len(desc) > max_desc_len:
                desc = desc[:max_desc_len] + "..."
            print(f"   📝 Description: {desc}")
            print(f"   🎯 Similarity: {doc['similarity_score']:.4f}")
            print(f"   🔢 ISBN13: {doc['isbn13']}")
            print("-" * 80)

# Initialize search
db_books = BookSearchFAISS(index, metadata, model)
print("FAISS search database ready!")

FAISS search database ready!


In [26]:
# Save FAISS index to disk
faiss.write_index(index, "book_faiss_index.index")

# Save metadata to disk (using JSON for readability)
with open("book_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("FAISS index and metadata saved to disk!")

FAISS index and metadata saved to disk!


In [27]:
# Load FAISS index from disk
index = faiss.read_index("book_faiss_index.index")

# Load metadata from disk
with open("book_metadata.json", "r") as f:
    metadata = json.load(f)

print(f"Loaded FAISS index with {index.ntotal} vectors and {len(metadata)} metadata entries")

# Initialize model (still needed for encoding queries)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize search
db_books = BookSearchFAISS(index, metadata, model)
print("FAISS search database loaded and ready!")

Loaded FAISS index with 5197 vectors and 5197 metadata entries
FAISS search database loaded and ready!


In [28]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=1)

print(f"🔍 FAISS Search results for: '{query}'\n")
db_books.print_results(docs)

🔍 FAISS Search results for: 'A book to teach children about nature'

1. 📚 Baby Einstein: Neighborhood Animals
   👤 Authors: Marilyn Singer;Julie Aigner-Clark
   🏷️  Category: Juvenile Fiction
   ⭐ Rating: 3.89 | 📄 Pages: 16.0
   📝 Description: Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The com...
   🎯 Similarity: 0.5306
   🔢 ISBN13: 9780786808069
--------------------------------------------------------------------------------
